# Cosheaf Homology with Discrete Morse Theory & the Mayer-Vietoris Sequence

Over the course of this project, we have defined cosheaves on simplicial complexes and explained how each has a sequence of homology groups. We have focused in particular on how an analog of the Mayer-Vietoris sequence that exists in simplicial homology persists in the context of cosheaves. To simplify calculations, we have also investigated how discrete Morse theoretic techniques may be adapted to this context.

In this notebook, we will provide some software implementations of relevant ideas from the course, including a definition of simplicial complexes and chain complexes, which is where we will begin. We will look at how discrete Morse theory in some sense simplifies this calculation, before essentially repeating the process for cosheaves. We conclude with an examination of the Mayer-Vietoris sequence from these perspectives.

Note that all of the code written in this notebook was written explicitly for (and over the duration of) the mini project for Computational Algebraic Topology 2024/2025.

## 1 Simplicial Complexes and Homology

We introduce a class capturing most of the salient features of a simplicial complex.

### 1.1 Simplicial Complexes

In [24]:
from functools import reduce
from copy import copy

Simplex = frozenset

def union(sets):
    return reduce(lambda x, y : x.union(y), sets, set())

def all_satisfy(predicate, iterable):
    return all(map(predicate, iterable))

def faces(simplex):
    if len(simplex) <= 1:
        return { }
    
    return set(map(lambda v : simplex - { v }, simplex))

def subsimplices(simplex):
    if not simplex:
        return {}
    
    return union(map(lambda face : subsimplices(face), faces(simplex))).union({ simplex })

simp_str = lambda simplex : f"{sorted(simplex)}"
dim = lambda simplex : len(simplex) - 1

In [25]:
class SimplicialComplex:
    def __init__(self, simplices: set[Simplex], no_assert=False):
        if not no_assert:
            included = lambda simplex : simplex in simplices
            faces_included = lambda simplex : all_satisfy(included, faces(simplex))
            assert(all_satisfy(faces_included, simplices))
        
        self.simplices = simplices
    
    def vertices(self):
        return union(self.simplices)
    
    def __contains__(self, simplex: Simplex) -> bool:
        return simplex in self.simplices
    
    def __iter__(self):
        return self.simplices.__iter__()

    @staticmethod
    def generated_by(simplices: set[Simplex]):
        all_simplices = union(map(lambda simplex : subsimplices(simplex), simplices))
        return SimplicialComplex(all_simplices)
    
    @staticmethod
    def ball(n: int):
        interior = Simplex(range(n + 1))
        return SimplicialComplex.generated_by({interior})
    
    @staticmethod
    def sphere(n: int):
        ball = SimplicialComplex.ball(n + 1)
        return SimplicialComplex(ball.simplices - { max(ball.simplices) })
    
    def __repr__(self) -> str:
        n = max(self.vertices()) ** 2 if self.simplices else 0
        sorted_simplices = sorted(self.simplices, key=simp_str)
        return "SimplicialComplex(simplices:" + ", ".join(map(simp_str, sorted_simplices)) + ")"
    
    # Not a super useful operation but should make certain constructions (e.g disjoint union or wedge sums) easier.
    def glue(self, other_complex, along=None):
        shift = (0 if not self.vertices() else max(self.vertices())) - (0 if not other_complex.vertices() else min(other_complex.vertices())) + 1
        shift_simplex = lambda simplex : Simplex(map(lambda x : x + shift, simplex))

        shifted_complex = SimplicialComplex(set(map(shift_simplex, other_complex.simplices)))

        sum_complex = SimplicialComplex(self.simplices.union(shifted_complex.simplices))

        if not along:
            return sum_complex
        
        along = { key : value + shift for key, value in along.items() }

        modify_simplex = lambda simplex : Simplex({ along[v] if v in along else v for v in simplex })
        glued_simplices = set(map(lambda simplex : modify_simplex(simplex), sum_complex.simplices))
        return SimplicialComplex(glued_simplices).cleaned_vertex_labels()

    def wedge(self, other_complex):
        return self.glue(other_complex, along = { 0 : 0 })

    def clean_vertex_labels(self) -> None:
        vertices = sorted(self.vertices())
        dict = { y : x for x, y in enumerate(vertices) }
        cleaned_simplices = set(map(lambda simplex : Simplex(map(lambda vertex : dict[vertex], simplex)), self.simplices))
        self.simplices = cleaned_simplices

    def cleaned_vertex_labels(self):
        c = copy(self)
        c.clean_vertex_labels()
        return c

SC = SimplicialComplex

### 1.2 Chain Complexes and Homology

We encode the chain complexes as a sequence of matrices representing the boundary operators. Implicity, we take our base field to be $\mathbb R$ in this notebook and therefore consider $C_\bullet(K, \mathbb R)$ in this subsection.
First we'll write some code to help find a basis for the homology of a chain complex given maps 
$$\cdots \to C_{n + 1} \xrightarrow{\partial_{n + 1}} C_n \xrightarrow{\partial_n} C_{n - 1} \to \cdots .$$

We will make use of the inner space structure on $\mathbb R^n$ via the isomorphism $H_n(C_\bullet, \partial_\bullet) = \ker \partial_n / {\rm im} \, \partial_{n + 1} \cong ({\rm im}\, \partial_{n + 1})^\bot$, where the orthogonal complement is taken in the ambient space $\ker \partial_n$.
Any integer manipulations below are to try to ensure that the resulting basis has integer coefficients, so that the vectors are a bit more geometrically interpretable as combinations of simplices in the underlying simplicial complex.

In [23]:
too_small = lambda m : np.all(np.isclose(np.array(m).astype(float), 0.0)) # the bugs I have chased due to `.is_zero_matrix` not being sensitive to floating point errors...

from sympy import Matrix, BlockMatrix, lcm
import numpy as np

def orthogonalize(A : np.array) -> tuple[Matrix, Matrix]:
    """
    Does not handle empty matrices very well! User beware!
    """
    dtype = int if all(x.is_integer for x in A) else float

    A = Matrix(A)
    Q = Matrix(np.identity(A.cols, dtype=dtype))

    for i in range(A.cols):
        Q_term = Matrix(np.identity(A.cols, dtype=dtype))
        
        col = A.col(i)

        for j in range(i):
            old_col = (A * Q).col(j)
            Q_term[j, i] = -col.dot(old_col) / old_col.dot(old_col) if not too_small(old_col) else 0

        if dtype == int:
            lcd = lcm([ x.denominator for x in (A * Q_term)[:, i]])
            Q_term[:, i] *= lcd # / g

        Q = Q * Q_term
        
    return A * Q, Q

def left_inverse(A: np.array) -> np.array:
    """
    Note the input matrix A is assumed to be injective, lest this task be impossible.
    """
    # this has taken me many attempts because I am embarrassingly bad at linear algebra.
    # write Q = AR where Q is orthogonal. Then Q^T(Q^T Q)^{-1} Q = Q^T(Q^T Q)^{-1} (AR) = I, so A has left inverse R Q^T(Q^T Q)^{-1}

    if not all(A.shape):
        return np.eye(A.shape[1], A.shape[0]) # i.e if A is the 0 map from a 0 dimensional space then sure the identity is a right inverse since 0 a

    A = Matrix(A)

    Q, R = orthogonalize(A)

    D = (Q.transpose() @ Q).inv() @ Q.transpose()

    return np.array(R @ D).astype(float)

def right_inverse(A: np.array) -> np.array:
    if not all(A.shape): # if the matrix is empty
        return np.identity(A.shape[0])
    
    A = Matrix(A)

    return left_inverse(A.transpose()).transpose()


def null_space(A: np.array):
    A = Matrix(A)
    ker = A.nullspace()

    dtype = int if all(x.is_integer for x in A) else float

    return Matrix.hstack(*[
        (col * lcm([ q.denominator for q in col ]) if dtype == int else col) for col in ker
    ])

def homology_basis(boundary_n: np.array, boundary_n_plus_one: np.array, degree_n: int) -> np.array:
    boundary_n = Matrix(boundary_n)
    boundary_n_plus_one = Matrix(boundary_n_plus_one)

    ker = null_space(boundary_n)

    img, _ = orthogonalize(boundary_n_plus_one)

    if not any(ker.shape):
        return np.zeros((degree_n, 0))
    
    aug_matrix = Matrix(BlockMatrix([img, ker])) if any(img.shape) else ker
    orthog, _ = orthogonalize(aug_matrix)

    return np.array(Matrix.hstack(*[
        orthog.col(i) for i in range(img.cols, orthog.cols) if not too_small(orthog.col(i))
    ])).astype(float).reshape((degree_n, -1))

We will ensure that the boundary maps defined satisfy $\partial_{n - 1} \circ \partial_n = 0$ for each $n$, to try to more faithfully represent the mathematical object we are attempting to capture.

In [4]:
from sympy import latex

rank = lambda m : 0 if 0 in m.shape else np.linalg.matrix_rank(m)

class ChainComplex:
    def __init__(self, degrees: tuple[int], boundary_maps: tuple[np.array]) -> None:
        self.__degrees = degrees
        self.__boundary_maps = [np.array(x) for x in boundary_maps]
        self.test_dims()
        self.test_square_to_zero()
        self.should_display_with_matrices = False
    
    def test_dims(self):
        assert all_satisfy(
            lambda i : self.boundary_map(i).shape == (self.degree(i - 1), self.degree(i)),
            range(self.max_dim() + 2)
        ), f"Boundary maps have unexpected shapes {[ self.boundary_map(i).shape for i in range(self.max_dim() + 2) ]} for degrees {[self.degree(i) for i in range(self.max_dim() + 2)]}"

    def test_square_to_zero(self):
        assert all_satisfy(
            lambda i : np.all(np.isclose(self.boundary_map(i - 1) @ self.boundary_map(i), 0.0)),
            range(self.max_dim() + 2)
        ), f"Boundary maps do not square to zero! {[self.boundary_map(i - 1) @ self.boundary_map(i) for i in range(self.max_dim())]} \n\n\n\nhere are the maps:\n\n\n\n {self.__boundary_maps}"

    @staticmethod
    def builder(degrees: tuple[int], boundary_builder):
        return ChainComplex(
            degrees,
            list(map(boundary_builder, degrees))
        )

    def degree(self, i: int) -> int:
        if 0 <= i and i < len(self.__degrees):
            return self.__degrees[i]
        return 0
    
    def max_dim(self) -> int:
        return len(self.__degrees) - 1
    
    def boundary_map(self, i: int) -> np.array:
        if 0 < i and i < self.max_dim() + 1:
            return self.__boundary_maps[i]
        if i == self.max_dim() + 1:
            return np.zeros((self.degree(i - 1), 0))
        if i == 0:
            return np.zeros((0, self.degree(0)))

        return np.zeros((0, 0))

    def sum(self, other_complex):
        max_i = max(self.max_dim(), other_complex.max_dim())

        first_diag = lambda i : self.boundary_map(i)
        second_diag = lambda i : other_complex.boundary_map(i)

        up_right = lambda i : np.zeros((
            self.degree(i - 1), other_complex.degree(i)
        ))
        bot_left = lambda i : np.zeros((
            other_complex.degree(i - 1), self.degree(i)
        ))
        
        sum_degrees = [ self.degree(i) + other_complex.degree(i) for i in range(max_i + 1)]
        sum_maps = [
            np.block([
                [first_diag(i),  up_right(i)],
                [bot_left(i),    second_diag(i)]
            ]) for i in range(max_i + 1)
        ]

        return ChainComplex(
            degrees=sum_degrees,
            boundary_maps=sum_maps,
        )
    
    def homology_basis(self, n: int):
        return homology_basis(boundary_n=self.boundary_map(n), boundary_n_plus_one=self.boundary_map(n + 1), degree_n=self.degree(n))
    
    def betti_number(self, n: int) -> int:
        return self.degree(n) - rank(self.boundary_map(n)) - rank(self.boundary_map(n + 1))

    def display_with_matrices(self, val: bool=True):
        self.should_display_with_matrices = val
        return self

    def _repr_latex_(self) -> str:
        arrow = lambda i : (latex(Matrix(self.boundary_map(i))) if all(self.boundary_map(i).shape) else '') if self.should_display_with_matrices else "\\partial_" + str(i)
        space = lambda i : "\\mathbb R^{" + str(self.degree(i)) if self.degree(i) else "{0"

        return f"$$\\cdots \\to 0 \\to " + f"".join([
            space(i) + "} \\xrightarrow{" + arrow(i) + "}" for i in range(self.max_dim(), 0, -1)
        ]) + space(0) + "} \\to 0 \\to \\cdots$$"

    def __repr__(self) -> str:
        return " --> ".join(reversed([ str(x) for x in self.__degrees ]))


In [5]:
# order of a simplex in another [σ : τ]
def order(sigma: Simplex, tau: Simplex) -> int:
    if sigma not in faces(tau):
        return 0
    return 1 if sorted(tau).index(min(tau - sigma)) % 2 == 0 else -1

def _boundary_map(complex: SimplicialComplex, n: int) -> np.array:
    # bucket simplices by dimension and sort them in a semi-reasonable way for prettiness
    domain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, complex.simplices), key=simp_str)
    codomain_simplices = sorted(filter(lambda simplex : dim(simplex) == n - 1, complex.simplices), key=simp_str)

    rows = len(codomain_simplices)
    cols = len(domain_simplices)

    if rows == 0 and cols == 0:
        return np.zeros((0, 0))
    if rows == 0:
        return np.zeros((0, cols))
    if cols == 0:
        return np.zeros((rows, 0))
        
    return np.array([
        [ order(codomain_simplices[i], domain_simplices[j]) for j in range(cols) ] for i in range(rows)
    ])

def chain_complex(K: SimplicialComplex) -> ChainComplex:
    n = max(map(dim, K)) + 1

    return ChainComplex(
        degrees=[ sum([ dim(s) == i for s in K ]) for i in range(n) ],
        boundary_maps= [ _boundary_map(K, i) for i in range(n) ]
    )

### 1.2.1 Example: Homology of a Torus

As a first example, we will look at the homology of the torus $S^1 \times S^1$. First we compute the relevant chain complex $C_\bullet(S^1 \times S^1; \mathbb R)$, using the simplicial complex obtained by cutting each square in a $3 \times 3$ grid into $2$ triangles and then gluing opposite sides of the grid together.

In [6]:
torus_to_line = lambda x, y : 3 * (x % 3) + (y % 3)

T = SimplicialComplex.generated_by(
    {
        Simplex({ torus_to_line(x, y), torus_to_line(x + 1, y), torus_to_line(x + 1, y + 1) }) for x in range(3) for y in range(3)
    }.union({
        Simplex({ torus_to_line(x, y), torus_to_line(x, y + 1), torus_to_line(x + 1, y + 1) }) for x in range(3) for y in range(3)
    })
)

C = chain_complex(T)
C

18 --> 27 --> 9

Then we compute its Betti number in degrees 0, 1, and 2.

In [26]:
for i in range(3):
    print(f"The {i}-th homology space of the torus is {C.betti_number(i)} dimensional.")

The 0-th homology space of the torus is 1 dimensional.
The 1-th homology space of the torus is 0 dimensional.
The 2-th homology space of the torus is 0 dimensional.


This is thankfully what we expect!

### 1.3 Simplifying with Discrete Morse Theory

Pairing simplices into an acyclic partial matching leaves behind a small set of critical simplices that may be seen as generators of a Morse chain complex, whose homology agrees with the ordinary one. We define a partial matching here and implement the algorithm suggested in the course notes to build its Morse complex.

In [27]:
from functools import lru_cache

class SimplexPair:
    def __init__(self, source: Simplex, target: Simplex) -> None:
        assert(source.issubset(target) and len(target - source) == 1)
        self.source = source
        self.target = target
    
    def both(self) -> set[Simplex]:
        return { self.source, self.target }
    
    def __repr__(self) -> str:
        return f"({simp_str(self.source)} ◃ {simp_str(self.target)})"

    def __eq__(self, value: object) -> bool:
        return value.__repr__() == self.__repr__()

    def __hash__(self) -> int:
        return self.__repr__().__hash__()

class PartialMatching:
    def __init__(self, complex: SimplicialComplex, pairs: set[SimplexPair], no_assert=False) -> None:
        if not no_assert:
            # no simplex belongs to more than one pair
            assert(all_satisfy(lambda pair : all_satisfy(lambda other_pair : pair == other_pair or pair.both().isdisjoint(other_pair.both()), pairs), pairs))
            assert(union(map(lambda pair : pair.both(), pairs)).issubset(complex.simplices))

        self.pairs = pairs
        self.complex = complex

    def sources(self) -> set[Simplex]:
        return { pair.source for pair in self.pairs }
    
    def targets(self) -> set[Simplex]:
        return { pair.target for pair in self.pairs }
    
    def simplices(self) -> set[Simplex]:
        return union([pair.both() for pair in self.pairs])

    def critical(self) -> set[Simplex]:
        return self.complex.simplices - union([pair.both() for pair in self.pairs])
    
    def restrict(self, complex: SimplicialComplex):
        return PartialMatching(complex, set( [ p for p in self.pairs if p.source in complex ]))

    def __getitem__(self, simplex: Simplex) -> Simplex:
        for pair in self.pairs:
            if pair.source == simplex:
                return pair.target
        return None
    
    def __contains__(self, simplex: Simplex) -> bool:
        return simplex in union([ pair.both() for pair in self.pairs ])
    
    def __repr__(self) -> str:
        return ", ".join([ pair.__repr__() for pair in self.pairs ])
    
    def __paths(self, origin: Simplex) -> list[Simplex]:
        if not self.pairs:
            return [[]]

        possible_faces = set(faces(origin)).intersection(self.sources())
        new_origin = lambda face : self[face]
        smaller_matching = lambda face : PartialMatching(self.complex, self.pairs - { SimplexPair(face, new_origin(face)) }, no_assert=True)

        return [
            [face, new_origin(face)] + p for face in possible_faces for p in smaller_matching(face).__paths(new_origin(face))
        ] + [[]]

    @lru_cache(maxsize=None)
    def paths(self) -> set[tuple]:
        length_bucketed_matchings = [
            PartialMatching(self.complex, set(filter(lambda pair : len(pair.source) - 1 == n, self.pairs)), no_assert=True) for n in range(max(map(len, self.complex)))
        ]

        paths_as_list: list[list] = sum([
            length_bucketed_matchings[len(sigma) - 2].__paths(origin=sigma) for sigma in self.complex if len(sigma) > 1 
        ], start=[])
        
        return { tuple(p) for p in paths_as_list }
    
    @lru_cache(maxsize=None)
    def critical_paths(self) -> set[tuple]:
        # print("not yet cached")
        length_bucketed_matchings = [
            PartialMatching(self.complex, set(filter(lambda pair : len(pair.source) - 1 == n, self.pairs)), no_assert=True) for n in range(max(map(len, self.complex)))
        ]

        paths_as_list: list[list] = sum([
            length_bucketed_matchings[len(sigma) - 2].__paths(origin=sigma) for sigma in self.critical() if len(sigma) > 1 
        ], start=[])
        
        return { tuple(p) for p in paths_as_list if not p or not set(faces(p[-1])).isdisjoint(self.critical()) }


### 1.3.1 Note on Complexity

It's worth mentioning here that the algorithm we have provided to find paths is not particularly quick: it generates the paths originating at a simplex $\sigma$ recursively by adding a pair $\alpha < \beta$ in $\Sigma$ that borders $\sigma$ to the path, and then considering all paths beginning at a vertex bordering $\beta$ using pairs in $\Sigma \setminus \{ \alpha < \beta \}$. Thus if $\Sigma$ contains $n$ pairs and the maximal dimension of a vertex in the simplicial complex is $d$, we can bound our complexity by a recurrence of the form $f(n) \leq d f(n - 1)$, which implies at worst exponential growth.

There are some simplifications we have made, for instance by noting that in our application we need only consider paths that connect critical simplices, but empirically this seems to be the slowest part of generating the Morse complex.
A benefit of this is that it needs only to be done once: thereafter all spaces tend to be small and all matrix operations are quick.

The code used to generate acyclic partial matchings is comparatively faster: it loops over simplices twice and after each inner loop either matches two simplices together or marks a simplex as critical, so that each loop runs at most $n$ times, where $n$ is the number of simplices in the initial complex. Since we do sort the list of simplices in the outer loop, we should technically report that the complexity of this function is $O(n^2 \log n)$, since the set operations in the inner loop are fast and require time proportional to the number of faces of a simplex, which is at most logarithmic in the number of simplices anyhow.

In [9]:
def acyclic_partial_matching(complex: SimplicialComplex, extra_conditions=lambda _ : True) -> PartialMatching:
    unassigned_simplices = copy(complex.simplices)
    pairs: set[SimplexPair] = set()

    while unassigned_simplices:
        found_good_pair = False

        for simplex in copy(unassigned_simplices):
            unassigned_faces = set(faces(simplex)).intersection(unassigned_simplices)
            
            if not found_good_pair and len(unassigned_faces) == 1 and extra_conditions(SimplexPair(source=min(unassigned_faces), target=simplex)):
                face = unassigned_faces.pop()
                                
                pairs.add(SimplexPair(face, simplex))
                unassigned_simplices.remove(simplex)
                unassigned_simplices.remove(face)
                
                found_good_pair = True
        
        if not found_good_pair:
            crit = min(sorted(unassigned_simplices, key=len))
            unassigned_simplices.remove(crit)
    
    return PartialMatching(complex, pairs)

In [10]:
def weight(path: list[Simplex]):
    terms = [ 
            -order(path[i], path[i + 1]) if i % 2 == 0 
        else order(path[i + 1], path[i]) 
        for i in range(len(path) - 1 ) 
    ]
    
    return reduce(lambda x, y : x * y, terms, 1)

def morse_order(paths, sigma: Simplex, tau: Simplex):
    return order(sigma, tau) + sum([ order(p[0], tau) * weight(p) * order(sigma, p[-1]) for p in paths if p ])

def _morse_boundary_map(matching: PartialMatching, n: int):
    critical_simplices = matching.critical()

    # bucket simplices by dimension and sort them in a semi-reasonable way for prettiness
    domain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, critical_simplices), key=simp_str)
    codomain_simplices = sorted(filter(lambda simplex : dim(simplex) == n - 1, critical_simplices), key=simp_str)

    rows = len(codomain_simplices)
    cols = len(domain_simplices)

    if rows == 0 and cols == 0:
        return np.zeros((0, 0))
    if rows == 0:
        return np.zeros((0, cols))
    if cols == 0:
        return np.zeros((rows, 0))
    
    all_paths = matching.critical_paths()

    # print(all_paths)

    return np.array([
        [ morse_order(all_paths, sigma, tau) for tau in domain_simplices ] for sigma in codomain_simplices
    ])

def morse_chain_complex(Σ: PartialMatching) -> ChainComplex:
    n = max(map(dim, Σ.critical())) + 1

    return ChainComplex(
        degrees=[ sum([ dim(s) == i for s in Σ.critical() ]) for i in range(n) ],
        boundary_maps= [ _morse_boundary_map(Σ, i) for i in range(n) ]
    )

### 1.3.2 Example: Morse and Simplicial Chain Complexes for the $k$-Sphere

To demonstrate the tremendous reduction in the size of the chain complexes afforded by discrete Morse theoretic techniques, we compute the homology of the $k$-sphere for $k = 4$ (though feel free to download the notebook play around with the value of $k$ in the following cell!) and observe the dimensions of the vector spaces in each chain complex and how large the matrices between them ordinarily are. The Morse complex, meanwhile, is *much* smaller.

Still, as we mentioned earlier, the computation of all the $\Sigma$-paths slows down as $k$ grows.

In [28]:
k = 4
K = SC.sphere(k)

Σ = acyclic_partial_matching(K)

C = chain_complex(K)
M = morse_chain_complex(Σ)

print(f"Ordinary chain complex corresponding to the {k}-sphere:")
display(C.display_with_matrices(True))
print(f"Morse chain complex corresponding to the {k}-sphere:")
display(M.display_with_matrices(True))

Ordinary chain complex corresponding to the 4-sphere:


6 --> 15 --> 20 --> 15 --> 6

Morse chain complex corresponding to the 4-sphere:


1 --> 0 --> 0 --> 0 --> 1

Quite a bit smaller!

## 2 Cosheaves and Cosheaf Homology

Having seen how simplicial homology may be computed, we turn our attention to homology with coefficients in a cosheaf. We begin by defining a cosheaf sitting over a simplicial complex.

### 2.1 Cosheaves

While a cosheaf is defined as a contravariant functor from a simplicial complex to the category of vector spaces (over a base field), we are not really able to capture much of this structure elegantly in Python. Instead we simply require a mapping from a pair of simplices to a matrix. We do provide a few simple examples of cosheaves, though, and a way to construct more complicated ones via sums.

In [12]:
class Cosheaf:
    def __init__(self, complex: SimplicialComplex, corestriction) -> None:
        """
        `corestriction` should be a function taking a pair of simplices (`sigma`, `tau`) with `tau` a subset of `sigma` and returning a matrix.
        """
        self.complex = complex
        self.corestriction = corestriction                

    @staticmethod
    def skyscraper(complex: SimplicialComplex, simplex: Simplex):
        return Cosheaf(
            complex,
            lambda s, t : np.eye(int(t == simplex), int(s == simplex))
        )
    
    def sum(self, other):
        """
        Assumes `self` and `other` are cosheaves over a common `SimplicialComplex`.
        """
        def sum_corestriction(s, t):
            A = self.corestriction(s, t)
            B = other.corestriction(s, t)

            return np.block([
                [A, np.zeros((A.shape[0], B.shape[1]))],
                [np.zeros((B.shape[0], A.shape[1])), B]
            ])

        return Cosheaf(self.complex, corestriction=sum_corestriction)
    
    @staticmethod
    def constant(complex: SimplicialComplex, n: int):
        return Cosheaf(
            complex=complex, corestriction=lambda x, y : np.identity(n)
        )
    
    def stalk_dimension(self, simplex):
        return rank(self(simplex, simplex))

    def __call__(self, bigger: Simplex, smaller: Simplex) -> np.array:
        if not smaller.issubset(bigger):
            return None
        return self.corestriction(bigger, smaller)

### 2.2 Cosheaf Homology

Once we have a cosheaf defined, we may define the complex of chains with coefficients in the cosheaf, as we did in the body of the report. 
The homology of the resulting complex is what interests us.

In [13]:
def _cosheaf_boundary_map(cosheaf: Cosheaf, n: int) -> np.array:
    complex = cosheaf.complex

    domain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, complex.simplices), key=simp_str)
    codomain_simplices = sorted(filter(lambda simplex : dim(simplex) == n - 1, complex.simplices), key=simp_str)

    rows = sum([ cosheaf.stalk_dimension(simplex) for simplex in codomain_simplices ])
    cols = sum([ cosheaf.stalk_dimension(simplex) for simplex in domain_simplices ])

    if rows == 0 and cols == 0:
        return np.zeros((0, 0))
    if rows == 0:
        return np.zeros((0, cols))
    if cols == 0:
        return np.zeros((rows, 0))

    return np.block([
        [ order(tau, sigma) * cosheaf.corestriction(sigma, tau) for sigma in domain_simplices] for tau in codomain_simplices
    ])

def cosheaf_chain_complex(cosheaf: Cosheaf) -> ChainComplex:
    n = max(map(dim, cosheaf.complex)) + 1

    return ChainComplex(
        degrees= [ sum([ cosheaf.stalk_dimension(simplex) for simplex in cosheaf.complex if dim(simplex) == i ]) for i in range(n) ],
        boundary_maps = [ _cosheaf_boundary_map(cosheaf, i) for i in range(n) ]
    )

### 2.2.1 Example: Cosheaves and their Homologies

We compute the homology of $S^{k_1} \vee S^{k_2}$ with coefficients over the constant $\mathbb R^2$ cosheaf $\mathcal C_{\mathbb R^2}$, the skyscraper ${\rm sk}^\tau$ (where $\tau$ is an $m$-simplex), and over the cosheaf $\mathcal C_{\mathbb R^2} \oplus {\rm sk}^\tau$.
We recall that we expect
$$ H_n(S^{k_1} \vee S^{k_2}; \mathcal C_{\mathbb R^2}) =  H_n(S^{k_1} \vee S^{k_2})^2,$$
where the latter is $\mathbb R$ in degrees $k_1$, $k_2$, and $0$, and otherwise vanishes. The skyscraper we expect to be $\mathbb R$ at $n = \dim \tau$, and $0$ elsewhere.
The homologies with coefficients in the sum cosheaf should be the sum of the homologies in the other two.

In [14]:
k1 = 5
k2 = 3
m = 4

K = SC.sphere(k1).wedge(SC.sphere(k2))

constant_cosheaf = Cosheaf.constant(
    K, n = 2
)

skyscraper_cosheaf = Cosheaf.skyscraper(
    K, simplex=set(range(m + 1))
)

C1 = cosheaf_chain_complex(constant_cosheaf)
C2 = cosheaf_chain_complex(skyscraper_cosheaf)
C3 = cosheaf_chain_complex(constant_cosheaf.sum(skyscraper_cosheaf))

print("Chain complexes:\n")

display(C1)
display(C2)
display(C3)

print(f"\nBetti Numbers for {k1}-sphere wedged {k2}-sphere via the constant ℝ^2 complex:", [ C1.betti_number(n) for n in range(k + 1) ])
print(f"Betti Numbers for skyscraper over {m} dimensional simplex:", [ C2.betti_number(n) for n in range(k + 1) ])
print(f"Betti Numbers for the sum of those two:", [ C3.betti_number(n) for n in range(k + 1) ])

Chain complexes:



14 --> 42 --> 80 --> 90 --> 62 --> 22

0 --> 1 --> 0 --> 0 --> 0 --> 0

14 --> 43 --> 80 --> 90 --> 62 --> 22


Betti Numbers for 5-sphere wedged 3-sphere via the constant ℝ^2 complex: [2, 0, 0, 2, 0]
Betti Numbers for skyscraper over 4 dimensional simplex: [0, 0, 0, 0, 1]
Betti Numbers for the sum of those two: [2, 0, 0, 2, 1]


This is what we expect. As a hint for what's to come, note the degrees of the spaces in the chain complexes!

### 2.3 Discrete Morse Theory

As we did before in Section 1.3, a large reduction in space is afforded by translating everything over to a Morse complex. We have a bit more work to do to define the Morse chain complex with coefficients in a cosheaf, which we do as outlined in the report. Passing extra conditions into the `acyclic_partial_matching` function does not break the guarantee that the returned partial matching will be acyclic, since this is enforced by the condition that a pair is only added at some iteration when it is of the form $\alpha < \beta$ with $\alpha$ being the only unpaired face of $\beta$ at that iteration.

In [29]:
def cosheaf_coherent_acyclic_partial_matching(cosheaf: Cosheaf, extra_conditions=lambda _ : True) -> PartialMatching: # What a mouthful!
    is_invertible = lambda m : m.shape[0] == m.shape[1] and np.linalg.det(m) != 0
    pair_is_coherent = lambda pair : is_invertible(cosheaf(bigger=pair.target, smaller=pair.source))

    return acyclic_partial_matching(cosheaf.complex, extra_conditions=lambda pair : (pair_is_coherent(pair) and extra_conditions(pair)))

def cosheaf_weight(cosheaf: Cosheaf, path: list[Simplex]) -> np.array:
    terms = [
        -np.linalg.inv(cosheaf_order(cosheaf, t=path[i + 1], s=path[i])) if i % 2 == 0
        else cosheaf_order(cosheaf, t=path[i], s=path[i + 1])
        for i in range(len(path) - 1)
    ]

    start = np.identity(terms[-1].shape[0] if terms else 1)
    return reduce(np.matmul, reversed(terms), start)

def cosheaf_order(cosheaf: Cosheaf, s: Simplex, t: Simplex) -> np.array:
    return np.zeros((cosheaf.stalk_dimension(s), cosheaf.stalk_dimension(t))) if order(s, t) == 0 else order(s, t) * cosheaf(t, s)

def cosheaf_morse_order(cosheaf: Cosheaf, matching: PartialMatching, s: Simplex, t: Simplex) -> np.array:    
    return cosheaf_order(cosheaf, s, t) + sum([ 
        cosheaf_order(cosheaf, s, p[-1]) @ cosheaf_weight(cosheaf, p) @ cosheaf_order(cosheaf, p[0], t) for p in matching.critical_paths() if p
    ])

def _cosheaf_morse_boundary_map(cosheaf: Cosheaf, matching: PartialMatching, n: int):
    critical_simplices = matching.critical()

    # bucket simplices by dimension and sort them in a semi-reasonable way for prettiness
    domain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, critical_simplices), key=simp_str)
    codomain_simplices = sorted(filter(lambda simplex : dim(simplex) == n - 1, critical_simplices), key=simp_str)

    rows = sum([ cosheaf.stalk_dimension(simplex) for simplex in codomain_simplices ])
    cols = sum([ cosheaf.stalk_dimension(simplex) for simplex in domain_simplices ])

    if rows == 0 and cols == 0:
        return np.zeros((0, 0))
    if rows == 0:
        return np.zeros((0, cols))
    if cols == 0:
        return np.zeros((rows, 0))

    return np.block([
        [ cosheaf_morse_order(cosheaf, matching, tau, sigma) for sigma in domain_simplices ] for tau in codomain_simplices
    ])


def cosheaf_morse_chain_complex(cosheaf: Cosheaf, Σ: PartialMatching) -> ChainComplex:
    n = max(map(dim, Σ.critical())) + 1

    return ChainComplex(
        degrees= [ sum([ cosheaf.stalk_dimension(simplex) for simplex in Σ.critical() if dim(simplex) == i ]) for i in range(n) ],
        boundary_maps = [ _cosheaf_morse_boundary_map(cosheaf, Σ, i) for i in range(n) ]
    )

### Example 2.3.1

We repeat the computations from Example 2.2.1, only this time applying the code we've written to use discrete Morse theory to find smaller chain complexes whose homologies agree with the ones we calculated earlier.

In [16]:
k1 = 5
k2 = 3
m = 4

K = SC.sphere(k1).wedge(SC.sphere(k2))

constant_cosheaf = Cosheaf.constant(
    K, n = 2
)

skyscraper_cosheaf = Cosheaf.skyscraper(
    K, simplex=set(range(m + 1))
)

Σ1 = cosheaf_coherent_acyclic_partial_matching(constant_cosheaf)
Σ2 = cosheaf_coherent_acyclic_partial_matching(skyscraper_cosheaf)
Σ3 = cosheaf_coherent_acyclic_partial_matching(constant_cosheaf.sum(skyscraper_cosheaf))

C1 = cosheaf_morse_chain_complex(constant_cosheaf, Σ1)
C2 = cosheaf_morse_chain_complex(skyscraper_cosheaf, Σ2)
C3 = cosheaf_morse_chain_complex(constant_cosheaf.sum(skyscraper_cosheaf), Σ3)

print("Chain complexes:\n")

display(C1)
display(C2)
display(C3)

print(f"\nBetti Numbers for {k1}-sphere wedged {k2}-sphere via the constant ℝ^2 complex:", [ C1.betti_number(n) for n in range(k + 1) ])
print(f"Betti Numbers for skyscraper over {m} dimensional simplex:", [ C2.betti_number(n) for n in range(k + 1) ])
print(f"Betti Numbers for the sum of those two:", [ C3.betti_number(n) for n in range(k + 1) ])

Chain complexes:



2 --> 0 --> 2 --> 0 --> 0 --> 2

0 --> 1 --> 0 --> 0 --> 0 --> 0

4 --> 3 --> 2 --> 0 --> 0 --> 2


Betti Numbers for 5-sphere wedged 3-sphere via the constant ℝ^2 complex: [2, 0, 0, 2, 0]
Betti Numbers for skyscraper over 4 dimensional simplex: [0, 0, 0, 0, 1]
Betti Numbers for the sum of those two: [2, 0, 0, 2, 1]


Notice that the Morse chain complexes are much smaller than the ones we originally computed.

## 3 Mayer-Vietoris

In this section, we will apply the previous techinques to the case of Mayer-Veitoris sequences, in which we may decompose a simplicial complex $K$ as a union of two subcomplexes $L$ and $M$.
These decompositions induce a short exact sequence of chain complexes
$$ 0 \to C_\bullet(L \cap M) \to C_\bullet(L) \oplus C_\bullet(M) \to C_\bullet(K) $$
where the chains take values in some field or cosheaf.
This in turn, by the snake lemma, induces a long exact sequence of homology: this is the Mayer-Vietoris sequence.

### 3.1 Chain Maps

We first define a chain map as a collection of maps between $C_n$ and $D_n$ for two chain complexes $C_\bullet$ and $D_\bullet$ and check that the appropriate squares commute.
We also define a way to handle the important class of chain maps between simplicial chain complexes $C_\bullet(K) \to C_\bullet(L)$ that are induced by simplicial maps $K \to L$.
Homology is also functorial, so chain maps induce maps on homology, which we compute via the `on_homology` method.

To better explain the thought process there, we recall that a map $f : C_\bullet \to D_\bullet$ extends to a map on homology via $[x] \mapsto [f_n(x)]$ in degree $n$.
Recall that we have been viewing homology as the orthogonal complement $H_n(C) \cong ({\rm im} \, \partial_{n + 1})^\bot$ where the ambient space is $\ker \partial_n$.
Under this identification, $H_n(C)$ includes into $\ker \partial_n$ and therefore into $C_n$, and so we have the following diagram,  where $i_n^C$ is the inclusion $H_n(C) \hookrightarrow C_n$.

<!-- https://q.uiver.app/#q=WzAsNCxbMCwwLCJDX24iXSxbMSwwLCJEX24iXSxbMCwxLCJIX24oQykiXSxbMSwxLCJIX24oRCkiXSxbMiwwLCJpX25eQyJdLFswLDEsImZfbiJdLFszLDEsImlfbl5EIiwyXSxbMiwzLCJIX24oZikiLDIseyJzdHlsZSI6eyJib2R5Ijp7Im5hbWUiOiJkYXNoZWQifX19XV0= -->
<iframe class="quiver-embed" src="https://q.uiver.app/#q=WzAsNCxbMCwwLCJDX24iXSxbMSwwLCJEX24iXSxbMCwxLCJIX24oQykiXSxbMSwxLCJIX24oRCkiXSxbMiwwLCJpX25eQyJdLFswLDEsImZfbiJdLFszLDEsImlfbl5EIiwyXSxbMiwzLCJIX24oZikiLDIseyJzdHlsZSI6eyJib2R5Ijp7Im5hbWUiOiJkYXNoZWQifX19XV0=&embed" width="358" height="304" style="border-radius: 8px; border: none;"></iframe>

Now commutativity of this square means that $i_n^D \circ H_n(f) = f_n \circ i_n^C$, so we can obtain $H_n(f)$ by finding a left inverse $L$ to $i_n^D$; then $H_n(f) = L \circ f_n \circ i_n^C$.

In [17]:
class ChainMap:
    def __init__(self, domain: ChainComplex, codomain: ChainComplex, maps: tuple[np.array]) -> None:
        self.domain = domain
        self.codomain = codomain
        self.__maps = maps
        self.test_dims()        
        self.test_naturality()
    
    def test_dims(self):
        assert all_satisfy(
            lambda i : self.map(i).shape == (self.codomain.degree(i), self.domain.degree(i)),
            range(self.max_dim() + 1)
        ), f"Maps have unexpected shapes {[ x.shape for x in self.__maps ]} for degrees {[ (self.domain.degree(i), self.codomain.degree(i)) for i in range(self.max_dim() + 1) ]}"
    
    def test_naturality(self):
        assert all_satisfy(
            lambda i : np.array_equal(self.map(i - 1) @ self.domain.boundary_map(i), self.codomain.boundary_map(i) @ self.map(i)),
            range(self.max_dim() + 1)
        ), f"Commuting square does not commute :("

        
    def max_dim(self):
        return max(self.domain.max_dim(), self.codomain.max_dim())

    def map(self, i: int) -> np.array:
        if 0 <= i and i < len(self.__maps):
            return self.__maps[i]
        return np.zeros((0, 0))
    
    def __call__(self, i: int) -> np.array:
        return self.map(i)
    
    @staticmethod
    def builder(C: ChainComplex, D: ChainComplex, map_builder):
        return ChainMap(
            domain=C,
            codomain=D,
            maps=list(map(map_builder, range(max(C.max_dim(), D.max_dim()) + 1)))
        )

    @staticmethod
    def from_simplicial_map(f, K: SimplicialComplex, L: SimplicialComplex):
        domain = chain_complex(K)
        codomain = chain_complex(L)

        def define_map(n):
            domain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, K), key=simp_str)
            codomain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, L), key=simp_str)

            M = np.zeros((len(codomain_simplices), len(domain_simplices)))

            for i in range(len(domain_simplices)):
                s = domain_simplices[i]
                
                if dim(f(s)) == n:
                    M[codomain_simplices.index(f(s))][i] = 1
            
            return M
        
        return ChainMap.builder(domain, codomain, map_builder=define_map)
    
    @staticmethod
    def from_inclusion(C: ChainComplex, D: ChainComplex, domain_simplices: set[Simplex], codomain_simplices: set[Simplex], cosheaf: Cosheaf=None):
        sd = lambda simplex : cosheaf.stalk_dimension(simplex) if cosheaf else 1

        def define_map(n):
            ds = sorted([s for s in domain_simplices if dim(s) == n], key=simp_str)
            cs = sorted([t for t in codomain_simplices if dim(t) == n], key=simp_str)

            M = [
                [ np.identity(sd(sigma)) if sigma == tau else np.zeros((sd(tau), sd(sigma))) for sigma in ds ] for tau in cs
            ]

            if not M or not M[0]:
                return np.zeros(
                    (sum([ sd(t) for t in cs]), sum([sd(s) for s in ds]))
                )
            
            return np.block(M)
        
        return ChainMap.builder(C, D, define_map)
    
    @staticmethod
    def from_partial_matching_restriction(Σ: PartialMatching, L: SimplicialComplex, cosheaf: Cosheaf=None):
        """
        Needs Σ to be an L-compatible and a cosheaf-compatible partial matching.
        """
        domain_simplices = set(L.simplices).intersection(Σ.critical())
        codomain_simplices = Σ.critical()

        C = cosheaf_morse_chain_complex(cosheaf=cosheaf, Σ=Σ.restrict(L))
        D = cosheaf_morse_chain_complex(cosheaf=cosheaf, Σ=Σ)

        return ChainMap.from_inclusion(C, D, domain_simplices, codomain_simplices, cosheaf)
    
    @staticmethod
    def from_critical_map(f, M1: PartialMatching, M2: PartialMatching):
        domain = morse_chain_complex(M1)
        codomain = morse_chain_complex(M2)

        def define_map(n):
            domain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, M1.critical()), key=simp_str)
            codomain_simplices = sorted(filter(lambda simplex : dim(simplex) == n, M2.critical()), key=simp_str)

            A = np.zeros((len(codomain_simplices), len(domain_simplices)))

            for i in range(len(domain_simplices)):
                s = domain_simplices[i]
                
                if dim(f(s)) == n:
                    A[codomain_simplices.index(f(s))][i] = 1
            
            return A
        
        return ChainMap.builder(domain, codomain, map_builder=define_map)

    def on_homology(self, n: int) -> np.array:
        H1 = self.domain.homology_basis(n)
        H2 = self.codomain.homology_basis(n)
        
        A = self.map(n)

        return left_inverse(H2) @ A @ H1

### 3.2 Snake Lemma

A short exact sequence of chain complexes
$$ 0 \to A \xrightarrow{f_\bullet} B \xrightarrow{g_\bullet} C \to 0 $$
gives rise to the following long exact sequence of vector spaces
$$ \cdots \to H_n(A) \xrightarrow{H_n(f)} H_n(B) \xrightarrow{H_n(g)} H_n(C) \xrightarrow{\delta_n} H_{n - 1}(A) \xrightarrow{H_{n - 1}(f)} H_{n - 1}(B) \to \cdots, $$
where $H_n(f)$ and $H_n(g)$ are as we already described, and $\delta_n$ arises from the snake lemma. More specifically, $\delta [c] =: [a]$ is defined by
$f_{n - 1}(a) = \partial^B_n(b)$, where $g(b) = c$.

The fact that this all works is covered in the course, but we must implement it explicitly here. So we find a left inverse $L_n$ to $f_n$ and a right inverse $R_n$ to $g_n$.
These exist because exactness guarantees that $f_n$ is injective and $g_n$ is surjective.
We then note that defining $a = (L_n \circ \partial^B_n \circ R_n)(c)$ and $b = R_n(c)$ for $c \in C_n$, we obtain
$f(a) = \partial^B_n(b)$ and $g(b) = c$, as we had desired.

Now we have a diagram as below: finding a left inverse to $i_{n - 1}^A$ allows us to solve for $\delta_n$.
<!-- https://q.uiver.app/#q=WzAsNCxbMCwxLCJIX3tuIC0gMX0oQSkiXSxbMCwwLCJBX3tuIC0gMX0iXSxbMiwwLCJDX3tufSJdLFsyLDEsIkhfbihDKSJdLFsyLDEsIkxfbiBcXGNpcmMgXFxwYXJ0aWFsX25eQiBcXGNpcmMgUl9uIiwyXSxbMywyLCJpX25eQyIsMl0sWzAsMSwiaV97biAtIDF9XkEiXSxbMywwLCJcXGRlbHRhX24iLDAseyJzdHlsZSI6eyJib2R5Ijp7Im5hbWUiOiJkYXNoZWQifX19XV0= -->
<iframe class="quiver-embed" src="https://q.uiver.app/#q=WzAsNCxbMCwxLCJIX3tuIC0gMX0oQSkiXSxbMCwwLCJBX3tuIC0gMX0iXSxbMiwwLCJDX3tufSJdLFsyLDEsIkhfbihDKSJdLFsyLDEsIkxfbiBcXGNpcmMgXFxwYXJ0aWFsX25eQiBcXGNpcmMgUl9uIiwyXSxbMywyLCJpX25eQyIsMl0sWzAsMSwiaV97biAtIDF9XkEiXSxbMywwLCJcXGRlbHRhX24iLDAseyJzdHlsZSI6eyJib2R5Ijp7Im5hbWUiOiJkYXNoZWQifX19XV0=&embed" width="511" height="304" style="border-radius: 8px; border: none;"></iframe>

In [18]:
def connecting_homomorphism(f: ChainMap, g: ChainMap, n: int, verbose=False):
    """
    Note we assume that 0 --> f.domain --> f.codomain == g.domain --> g.codomain --> 0 is a short exact sequence of chain complexes.
    """
    if verbose:
        print("f(n - 1)")
        print(f(n - 1))
        print("\n")
        print(g(n))

    L = left_inverse(f(n - 1))
    R = right_inverse(g(n))
    
    M = left_inverse(f.domain.homology_basis(n - 1))

    return M @ L @ f.codomain.boundary_map(n) @ R @ g.codomain.homology_basis(n)

### 3.2.1 Why does the Morse Complex Help?

In the code above, we find a `left_inverse` of the map $f$ and `right_inverse` of the map $g$, which are chain maps. If we were to use the chain complexes $(C_\bullet(K; \mathcal C), \partial^{\mathcal C}_\bullet)$ as a source for this calculation, we would have to take inverses and multiply together matrices that are quite large when there are many simplices.
Indeed, multiplication of a $k \times m$ matrix by an $m \times n$ matrix is $O(kmn)$, and the method we use for finding left and right inverses relies on factorizing into orthogonal and upper diagonal matrices, which is also a costly operation.
More details on complexity of matrix operations may be found online <a href="https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Matrix_algebra">here</a> (also cited in the references of the report).

In any case, there is a clear impetus to reduce the size of any chain complexes where possible, and since we are only interested in our complexes up to quasi-isomorphism, there is no salient information lost by passing down to the Morse complex.

In [19]:
def sum_chain_maps(f: ChainMap, g: ChainMap, same_domain: bool=False, same_codomain: bool=False, mult: tuple[float, float]=(1., 1.)) -> ChainMap:
    a, b = mult
    
    if same_domain and same_codomain:
        return ChainMap.builder(
            f.domain,
            f.codomain,
            lambda n : a * f(n) + b * g(n)
        )
    if same_domain:
        map_builder_same_domain = lambda n : np.block([[a * f(n)], [b * g(n)]])
        return ChainMap.builder(
            f.domain,
            f.codomain.sum(g.codomain),
            map_builder_same_domain
        )
    if same_codomain:
        map_builder_same_codomain = lambda n : np.block([[a * f(n), b * g(n)]])
        return ChainMap.builder(
            f.domain.sum(g.domain),
            f.codomain,
            map_builder_same_codomain
        )
    map_builder_different = lambda n : np.block([
            [mult[0] * f(n), np.zeros((f(n).shape[0], g(n).shape[1]))],
            [np.zeros((g(n).shape[0], f(n).shape[1])), mult[1] * g(n)]
        ])

    return ChainMap.builder(
        f.domain.sum(g.domain),
        f.codomain.sum(g.codomain),
        map_builder_different
    )

We'll introduce a class here that encapsulates what it means to be a Mayer-Vietoris sequence: to be a long exact sequence arising from a chain complex that arises from inclusions of $L \cap M$ into $L$ and $M$ and then into $L \cup M$.

In [20]:
def mvsi(n, i):
    return 3 * n + (2 - i)

def isvm(n):
    return (n // 3, 2 - (n % 3))

class MayerVietoris:
    def __init__(self, part1, part2, intersection, union, chain_complex: ChainComplex) -> None:
        self.part1 = part1
        self.part2 = part2
        self.intersection = intersection
        self.union = union
        self._chain_complex = chain_complex
        self.show_matrices = False
        self.should_display_with_matrices = False

    def max_dim(self):
        n, _ = isvm(self._chain_complex.max_dim())
        return n

    def degree(self, n: int, i):
        return self._chain_complex.degree(mvsi(n, i))
    
    def map(self, n: int, i):
        return self._chain_complex.boundary_map(mvsi(n, i))
    
    def display_with_matrices(self):
        self.should_display_with_matrices = True
        return self
    
    @staticmethod
    def from_inclusions(alpha, beta, i, j, L: SimplicialComplex, M: SimplicialComplex):
        L_cap_M = SimplicialComplex(set(L.simplices).intersection(M.simplices), no_assert=True)
        L_cup_M = SimplicialComplex(set(L.simplices).union(M.simplices), no_assert=True)

        f = sum_chain_maps(alpha, beta, same_domain=True)
        g = sum_chain_maps(i, j, same_codomain=True, mult=(1, -1))
        
        max_dim = max(map(dim, L_cup_M.simplices))
        max_degree = 3 * max_dim + 2

        def degree(n, i):
            if i == 2:
                return g.codomain.betti_number(n)
            if i == 1:
                return g.domain.betti_number(n)
            if i == 0:
                return f.domain.betti_number(n)
            return None

        def get_map(n, i):
            if i == 2:
                return connecting_homomorphism(f, g, n)
            if i == 1:
                return g.on_homology(n)
            if i == 0:
                return f.on_homology(n)
            
        cc = ChainComplex(
            degrees=[ degree(*isvm(n)) for n in range(max_degree) ],
            boundary_maps=[ get_map(*isvm(n)) for n in range(max_degree) ]
        )

        return MayerVietoris(L, M, L_cap_M, L_cup_M, cc)

    @staticmethod
    def morse_cosheaf(L: SimplicialComplex, M: SimplicialComplex, cosheaf: Cosheaf) -> None:
        """
        Must be that L U M = cosheaf.complex.
        """
        L_cap_M = SimplicialComplex(simplices=L.simplices.intersection(M.simplices))

        L_compatible = lambda pair : ((pair.source in L) == (pair.target in L))
        M_compatible = lambda pair : ((pair.source in M) == (pair.target in M))
        Σ = cosheaf_coherent_acyclic_partial_matching(cosheaf, extra_conditions=lambda pair : L_compatible(pair) and M_compatible(pair))

        alpha = ChainMap.from_partial_matching_restriction(Σ.restrict(L), L_cap_M, cosheaf)
        beta = ChainMap.from_partial_matching_restriction(Σ.restrict(M), L_cap_M, cosheaf)
        i = ChainMap.from_partial_matching_restriction(Σ, L, cosheaf)
        j = ChainMap.from_partial_matching_restriction(Σ, M, cosheaf)

        return MayerVietoris.from_inclusions(alpha, beta, i, j, L, M)

    @staticmethod
    def simplicial(L: SimplicialComplex, M: SimplicialComplex) -> None:
        L_cap_M = SimplicialComplex(set(L.simplices).intersection(M.simplices), no_assert=True)
        L_cup_M = SimplicialComplex(set(L.simplices).union(M.simplices), no_assert=True)

        id = lambda x : x

        alpha = ChainMap.from_simplicial_map(id, L_cap_M, L)
        beta = ChainMap.from_simplicial_map(id, L_cap_M, M)
        i = ChainMap.from_simplicial_map(id, L, L_cup_M)
        j = ChainMap.from_simplicial_map(id, M, L_cup_M)

        return MayerVietoris.from_inclusions(alpha, beta, i, j, L, M) # (L, M, L_cap_M, L_cup_M, cc)
    

    def _repr_latex_(self) -> str:
        arrow_no_matrices = lambda n, i : f"H_{n}(f)" if i == 0 else "H_{n}(g)" if i == 1 else f"\\delta_{n}"
        arrow = lambda n, i : (latex(Matrix(self.map(n, i))) if all(self.map(n, i).shape) else '') if self.should_display_with_matrices else arrow_no_matrices(n, i)
        space = lambda n, i : "\\mathbb R^{" + str(self.degree(n, i)) if self.degree(n, i) else "{0"

        return "\\begin{align*}" + "\\\\ \\to".join([
            "".join([ "&" + (i * "&") + space(n, i) + "}\\xrightarrow{" + arrow(n, i) + "}" for i in range(3) ]) for n in range(self.max_dim(), -1, -1)
        ]) + "\\end{align*}"


    def __repr__(self) -> str:
        rows = [ " --> ".join([ str(self.degree(n, i)) for i in range(3) ]) for n in range(self.max_dim(), -1, -1) ]
        return " -->\n".join(rows)


### 3.2.2 Example

We decompose $K = \partial \Delta(3)$ as the union of $L$ and $M$ where $L = \Delta(2)$ and $M = \Lambda^3_3$, i.e everything but the face opposite $3$ of $\partial \Delta(3)$.
$L$ and $M$ intersect in $\partial \Delta(2)$.

We put a peculiar cosheaf over this simplicial complex for the purposes of a demonstration, written as the sum of a few skyscrapers and a constant cosheaf.
We then find the Mayer-Vietoris sequence associated to this decomposition, where we leverage the power of discrete Morse theory to find a smaller chain complex than the one we'd typically define to simplify calculation.

The code that does this lies is in the `MayerVietoris.morse_cosheaf` method, and finds an acyclic partial matching that is compatible with the cosheaf and both $L$ and $M$, in the sense we defined in the report.

In [21]:
K = SC.sphere(2)

L = SC.ball(2)
M = SimplicialComplex.generated_by(
    { f for f in faces(Simplex({0, 1, 2, 3})) if 3 in f }
)

cosheaf = reduce(lambda x, y : x.sum(y), [ Cosheaf.skyscraper(K, {v}) for v in [0, 1, 2] ], Cosheaf.constant(K, n = 0)).sum(Cosheaf.constant(K, 3)) # some wacky looking cosheaf!

MayerVietoris.morse_cosheaf(L, M, cosheaf) # .display_with_matrices() this looks a bit wonky with the align environment...

0 --> 0 --> 3 -->
3 --> 0 --> 0 -->
6 --> 12 --> 6

## 4. Possible Extensions

Aside from the routine cleaning of hastily written code and rooting out of whatever bugs may linger unnoticed, there are a number of possible directions we might take to better this notebook. 

<ul>
    <li> More diverse examples of cosheaves: This would have a primarily pedagogic benefit on the notebook. </li><br/>
    <li> More detailed analysis of algorithms: The code written for this project was written with the intent of being functional but not for being optimal. For this reason, the algorithms that were implemented were not carefully considered for efficiency, and it would be interesting and worthwhile to spend some time deciding which aspects of the program can be made faster, and which are fundamentally slow. As an example, it seems unlikely that the implementation of the computation of the boundary operator for Morse complexes is at present optimal. Investigating whether that is the case would be a useful extension.  </li><br/>
    <li> Applications to TDA: It would be an interesting extension of this project to research what cosheaves are frequently used in topological data analysis and to take an actual dataset, adapt the techinques here to be compatible with filtrations, and try to learn something meaningful about the dataset by computing its homology groups.
</ul>